In [ ]:
!pip install transformers

# Kindly add all your installations and versions if any in this cell.

In [20]:
import pandas as pd
import numpy as np
import spacy
import csv
from scipy import stats
from sklearn import linear_model

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import re
import matplotlib.pyplot as plt
import torch

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
INPUT_PATH="drive/MyDrive/NLP Ass3 stsbenchmark/"
def read_sts_csv(dataset_type="train", columns=['source', 'type', 'year', 'id', 'score', 'sent_a', 'sent_b']):
  path = INPUT_PATH + "sts-"+ dataset_type + ".csv"
  return pd.read_csv(path,sep="\t",header=None, names=columns,quoting=csv.QUOTE_NONE)

data_train = read_sts_csv("train") # create the train, dev and test dataframes
data_dev = read_sts_csv("dev") # create the train, dev and test dataframes
data_test = read_sts_csv("test") # create the train, dev and test dataframes


In [ ]:
from transformers import BartTokenizer, BartModel

tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartModel.from_pretrained('facebook/bart-base')


In [21]:
inputs = tokenizer("hi my", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state

In [22]:
last_hidden_states.shape


torch.Size([1, 4, 768])

In [ ]:
sent_a_tensor_train=[]
for i in data_train['sent_a']:
  input = tokenizer(i, return_tensors="pt")
  output = model(**inputs)
  hidden_state = outputs.last_hidden_state
  sent_a_tensor_train.append(hidden_state)

sent_b_tensor_train=[]
for i in data_train['sent_b']:
  input = tokenizer(i, return_tensors="pt")
  output = model(**inputs)
  hidden_state = outputs.last_hidden_state
  sent_a_tensor_train.append(hidden_state)


In [ ]:
sent_a_tensor_dev=[]
for i in data_dev['sent_a']:
  input = tokenizer(i, return_tensors="pt")
  output = model(**inputs)
  hidden_state = outputs.last_hidden_state
  sent_a_tensor_dev.append(hidden_state)

sent_b_tensor_dev=[]
for i in data_dev['sent_b']:
  input = tokenizer(i, return_tensors="pt")
  output = model(**inputs)
  hidden_state = outputs.last_hidden_state
  sent_b_tensor_dev.append(hidden_state)


In [ ]:
sent_a_tensor_test=[]
for i in data_test['sent_a']:
  input = tokenizer(i, return_tensors="pt")
  output = model(**inputs)
  hidden_state = outputs.last_hidden_state
  sent_a_tensor_test.append(hidden_state)

sent_b_tensor_test=[]
for i in data_test['sent_b']:
  input = tokenizer(i, return_tensors="pt")
  output = model(**inputs)
  hidden_state = outputs.last_hidden_state
  sent_b_tensor_test.append(hidden_state)


In [ ]:
# # Concatenate the tensors for all layers. We use `stack` here to
# # create a new dimension in the tensor.
# token_embeddings_all=[]
# for i in hidden_states_all:

#   token_embeddings = torch.stack(i, dim=0)
#   token_embeddings_all.append(token_embeddings)

#   # token_embeddings.size()

In [ ]:
token_embeddings.size()

In [ ]:
# # Remove dimension 1, the "batches".
# temp=[]
# for i in token_embeddings_all:
#   token_embeddings = torch.squeeze(token_embeddings, dim=1)
#   temp.append(token_embeddings)
# token_embeddings_all=temp
# # token_embeddings.size()

In [ ]:
# # Swap dimensions 0 and 1.
# temp=[]
# for i in token_embeddings_all:
#   token_embeddings = token_embeddings.permute(1,0,2)
#   temp.append(token_embeddings)
# token_embeddings_all=temp
# # token_embeddings.size()

In [ ]:
# Stores the token vectors, with shape [22 x 3,072]
token_vecs_cat_all=[]

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for i in sent_a_tensor_train:
  token_vecs_cat = []
  for token in i:
      
      # `token` is a [12 x 768] tensor

      # Concatenate the vectors (that is, append them together) from the last 
      # four layers.
      # Each layer vector is 768 values, so `cat_vec` is length 3,072.
      cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
      
      # Use `cat_vec` to represent `token`.
      token_vecs_cat.append(cat_vec)
  token_vecs_cat_all.append(token_vecs_cat)

  print ('Shape is: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))

In [ ]:
# # Stores the token vectors, with shape [22 x 768]
# token_vecs_cat_all = []

# # `token_embeddings` is a [22 x 12 x 768] tensor.

# # For each token in the sentence...
# for i in token_embeddings_all:
#   token_vecs_cat = []
#   for token in i:

#       # `token` is a [12 x 768] tensor

#       # Sum the vectors from the last four layers.
#       cat_vec = torch.sum(token[-4:], dim=0)
      
#       # Use `sum_vec` to represent `token`.
#       token_vecs_cat.append(cat_vec)
#   token_vecs_cat_all.append(token_vecs_cat)

#   print ('Shape is: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))

In [ ]:
# `hidden_states` has shape [13 x 1 x 22 x 768]

# `token_vecs` is a tensor with shape [22 x 768]
for i in hidden_states_all:
  token_vecs = i[-2][0]

  # Calculate the average of all 22 token vectors.
  sentence_embedding = torch.mean(token_vecs, dim=0)

In [ ]:
print ("Our final sentence embedding vector of shape:", sentence_embedding.size())

In [ ]:
for i, token_str in enumerate(tokenized_text):
  print (i, token_str)

In [ ]:
print('First 5 vector values for each instance of any word.')
print('')
print(token_vecs_cat)


In [ ]:
print(len(token_vecs_cat_all),len(data_train['sent_a']))

In [ ]:
for i in range(len(token_embeddings_all)):
  temp_vec=[]
  for token in token_embeddings_all[i]:
    for j in token.tolist():
      temp_vec.append(j)
  token_embeddings_all[i]=temp_vec

In [ ]:
sent_a_token_vecs_all=[]
sent_b_token_vecs_all=[]
for i in range(5749):
  sent_a_token_vecs_all.append(token_vecs_cat_all[i])
for i in range(5749,5749*2):
  sent_b_token_vecs_all.append(token_vecs_cat_all[i])
print(len(sent_a_token_vecs_all),len(sent_b_token_vecs_all))



In [ ]:
sent_vects_concat_all=[]
for i in range(5749):
  for j in sent_a_token_vecs_all[i]:
    sent_vects_concat_all.append(j)
  for k in sent_b_token_vecs_all[i]:
    sent_vects_concat_all.append(k)

In [ ]:
y_train, x_train = data_train['score'],sent_vects_concat_all



In [ ]:
regr = linear_model.LinearRegression()

y=regr.fit(x_train, y_train)
from scipy.stats import spearmanr
print()

In [ ]:
ypred=y.predict(x_train)

In [ ]:
print(ypred)
coef, p = spearmanr(ypred, y_train)

In [ ]:
print(coef)